In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

import cv2
import numpy as np

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
X_train, y_train = pd.read_pickle("data/x_train.pkl"), pd.read_pickle("data/y_train.pkl")
X_val, y_val = pd.read_pickle("data/x_val.pkl"), pd.read_pickle("data/y_val.pkl")

## Соберем датасеты и загрузим модель

In [4]:
from task_models import CatDogsData, BB_model
train_ds = CatDogsData(X_train['new_path'],X_train['new_bb'] ,y_train, transforms=True)
valid_ds = CatDogsData(X_val['new_path'],X_val['new_bb'],y_val)

In [5]:
batch_size = 16
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [6]:
model = BB_model().cuda()
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.006)
epochs = 50

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Обучим модель. Коэффициент С служит для приведения ошибки bb и классификации к схожим уровням

In [7]:
%%time
#for i in range(epochs):
for i in range(epochs):    
    model.train()
    total = 0
    sum_loss = 0
    C=1000
    
    for x, y_class, y_bb in train_dl:
        optimizer.zero_grad()
        batch = y_class.shape[0]
        x = x.cuda().float()
        y_class = y_class.cuda()
        y_bb = y_bb.cuda().float()
#         print(y_bb.shape)
#         print(y_class.shape)
        out_class, out_bb = model(x)
#         print(out_bb.shape)
#         print(out_class.shape)
        loss_class = F.cross_entropy(out_class, y_class, reduction="sum")
        loss_bb = F.l1_loss(out_bb, y_bb, reduction="none").sum(1)
        loss_bb = loss_bb.sum()
        loss = loss_class + loss_bb/C

        loss.backward()
        optimizer.step()
        total += batch
        sum_loss += loss.item()
    train_loss = sum_loss/total
    
    # Eval
    model.eval()
    val_total = 0
    val_sum_loss = 0
    correct = 0
    
    for x, y_class, y_bb in valid_dl:
        batch = y_class.shape[0]
        x = x.cuda().float()
        y_class = y_class.cuda()
        y_bb = y_bb.cuda().float()
        out_class, out_bb = model(x)
        loss_class = F.cross_entropy(out_class, y_class, reduction="sum")

        loss_bb = F.l1_loss(out_bb, y_bb, reduction="none").sum(1)
        loss_bb = loss_bb.sum()
        loss = loss_class + loss_bb/C
        _, pred = torch.max(out_class, 1)
        correct += pred.eq(y_class).sum().item()
        val_sum_loss += loss.item()
        val_total += batch
    val_loss = val_sum_loss/val_total
    val_acc = correct/val_total
    
    print("train_loss %.3f val_loss %.3f val_acc %.3f" % (train_loss, val_loss, val_acc))

train_loss 1.097 val_loss 1.139 val_acc 0.642
train_loss 0.891 val_loss 6.409 val_acc 0.611
train_loss 0.897 val_loss 0.899 val_acc 0.626
train_loss 0.863 val_loss 0.822 val_acc 0.659
train_loss 0.822 val_loss 0.887 val_acc 0.614
train_loss 0.836 val_loss 0.803 val_acc 0.654
train_loss 0.793 val_loss 0.814 val_acc 0.659
train_loss 0.787 val_loss 0.799 val_acc 0.661
train_loss 0.784 val_loss 0.807 val_acc 0.660
train_loss 0.777 val_loss 0.786 val_acc 0.661
train_loss 0.773 val_loss 0.784 val_acc 0.661
train_loss 0.774 val_loss 0.788 val_acc 0.679
train_loss 0.773 val_loss 0.789 val_acc 0.667
train_loss 0.763 val_loss 0.807 val_acc 0.653
train_loss 0.756 val_loss 0.768 val_acc 0.720
train_loss 0.748 val_loss 0.824 val_acc 0.649
train_loss 0.750 val_loss 0.748 val_acc 0.702
train_loss 0.731 val_loss 0.773 val_acc 0.703
train_loss 0.737 val_loss 0.751 val_acc 0.694
train_loss 0.724 val_loss 0.761 val_acc 0.705
train_loss 0.721 val_loss 0.735 val_acc 0.724
train_loss 0.715 val_loss 0.708 va

## Сохраним модель

In [8]:
model_path = "data/resnet34"
torch.save(model.state_dict(), model_path)